In [1]:
# Dependencies
import pandas as pd
import numpy as np

In [2]:
# Pulling CSVs into DataFrames
campaigns_df = pd.DataFrame(pd.read_csv('data/campaigns.csv'))
email_interactions_df = pd.DataFrame(pd.read_csv('data/email_interactions.csv'))
target_profiles_df = pd.DataFrame(pd.read_csv('data/target_profiles.csv'))
targeted_moviegoers_df = pd.DataFrame(pd.read_csv('data/targeted_moviegoers.csv'))
transaction_lines_df = pd.DataFrame(pd.read_csv('data/transaction_lines.csv'))
campaigns_df.head()

,campaign_id,campaign_description,campaign_date,campaign_target_group_count,campaign_control_group_count,campaign_has_control_group
0,22160,Campaign # 1,2019-01-01,1,NaN,False
1,22166,Campaign # 2,2019-01-03,1981,NaN,False
2,22173,Campaign # 3,2019-01-04,2648,NaN,False
3,22174,Campaign # 4,2019-01-04,3540,NaN,False
4,22175,Campaign # 5,2019-01-04,11194,NaN,False


In [3]:
## Converting Campaign Columns to Correct type

# Campaign Date to Datetime type
campaigns_df['campaign_date'] = pd.to_datetime(campaigns_df['campaign_date'])

# Campaign Control Group Count to int type
campaigns_df['campaign_control_group_count'] = campaigns_df['campaign_control_group_count'].fillna(0).astype(int)
campaigns_df.head()

,campaign_id,campaign_description,campaign_date,campaign_target_group_count,campaign_control_group_count,campaign_has_control_group
0,22160,Campaign # 1,2019-01-01,1,0,False
1,22166,Campaign # 2,2019-01-03,1981,0,False
2,22173,Campaign # 3,2019-01-04,2648,0,False
3,22174,Campaign # 4,2019-01-04,3540,0,False
4,22175,Campaign # 5,2019-01-04,11194,0,False


In [4]:
## Creating the 'campaign_unique_email_opens' column from email_interactions_df

# Filtering only 'email_open' interaction types and dropping duplicate opens by the same moviegoer
campaign_unique_email_opens_df = email_interactions_df.loc[email_interactions_df['email_interaction_type'] == 'email_open'].drop_duplicates(subset='moviegoer_id')

# Counted the number of unique moviegoers that opened emails grouping by campaign_id
campaign_unique_email_opens_df = campaign_unique_email_opens_df.groupby(['campaign_id']).count()

# Formatting DataFrame to merge with campaigns_df
campaign_unique_email_opens_df.drop(columns = ['email_interaction_type', 'email_interaction_date'], inplace=True)
campaign_unique_email_opens_df.rename(columns = {'moviegoer_id':'campaign_unique_email_opens'}, inplace=True)


# Merging campaign_unique_email_opens_df with campaigns_df on campaign_id
campaigns_summary_df = pd.merge(campaigns_df, campaign_unique_email_opens_df, left_on='campaign_id', right_index=True, how='left')
campaigns_summary_df['campaign_unique_email_opens'] = campaigns_summary_df['campaign_unique_email_opens'].fillna(0).astype(int)
campaigns_summary_df.head()


,campaign_id,campaign_description,campaign_date,campaign_target_group_count,campaign_control_group_count,campaign_has_control_group,campaign_unique_email_opens
0,22160,Campaign # 1,2019-01-01,1,0,False,0
1,22166,Campaign # 2,2019-01-03,1981,0,False,181
2,22173,Campaign # 3,2019-01-04,2648,0,False,177
3,22174,Campaign # 4,2019-01-04,3540,0,False,289
4,22175,Campaign # 5,2019-01-04,11194,0,False,386


In [5]:
## Creating the 'campaign_unique_link_clicks' using reformatted code from the 'campaign_unique_email_opens'

# Filtering only 'email_open' interaction types and dropping duplicate opens by the same moviegoer
campaign_unique_link_clicks_df = email_interactions_df.loc[email_interactions_df['email_interaction_type'] == 'link_click'].drop_duplicates(subset='moviegoer_id')

# Counted the number of unique moviegoers that opened emails grouping by campaign_id
campaign_unique_link_clicks_df = campaign_unique_link_clicks_df.groupby(['campaign_id']).count()

# Formatting DataFrame to merge with campaigns_df
campaign_unique_link_clicks_df.drop(columns = ['email_interaction_type', 'email_interaction_date'], inplace=True)
campaign_unique_link_clicks_df.rename(columns = {'moviegoer_id':'campaign_unique_link_clicks'}, inplace=True)

# Merging campaign_unique_email_opens_df with campaigns_df on campaign_id
campaigns_summary_df = pd.merge(campaigns_summary_df, campaign_unique_link_clicks_df, left_on='campaign_id', right_index=True, how='left')
campaigns_summary_df['campaign_unique_link_clicks'] = campaigns_summary_df['campaign_unique_link_clicks'].fillna(0).astype(int)
campaigns_summary_df.head()

,campaign_id,campaign_description,campaign_date,campaign_target_group_count,campaign_control_group_count,campaign_has_control_group,campaign_unique_email_opens,campaign_unique_link_clicks
0,22160,Campaign # 1,2019-01-01,1,0,False,0,1
1,22166,Campaign # 2,2019-01-03,1981,0,False,181,30
2,22173,Campaign # 3,2019-01-04,2648,0,False,177,83
3,22174,Campaign # 4,2019-01-04,3540,0,False,289,217
4,22175,Campaign # 5,2019-01-04,11194,0,False,386,94


In [6]:
# Changing the transaction_line_date column of the transaction_lines_df to datetime type
transaction_lines_df['transaction_line_date'] = pd.to_datetime(transaction_lines_df['transaction_line_date'])
transaction_lines_df.head()

,transaction_line_id,transaction_id,moviegoer_id,transaction_line_total,transaction_line_date,transaction_line_type,transaction_line_is_admission
0,32390599,21147938,325785,7.36,2019-02-01,box_office,True
1,32031963,21051876,41541,1.25,2019-01-04,other,NaN
2,32277876,21117137,47771,7.98,2019-01-21,concession,NaN
3,32363598,21140456,92925,6.42,2019-01-28,concession,NaN
4,32164860,21087930,524399,5.96,2019-01-13,concession,NaN


In [7]:
## Creating the campaign_target_total_spend column

# Merging targeted campaign info and moviegoer info to the transaction data
transactions_campaign_id_df = pd.merge(targeted_moviegoers_df, transaction_lines_df, left_on='moviegoer_id', right_on='moviegoer_id', how='left')

# Trimming the campaigns_df to merge campaign start dates into the transactions dataframe
campaigns_trimmed_df = campaigns_df.drop(columns = ['campaign_description', 'campaign_target_group_count', 'campaign_control_group_count', 'campaign_has_control_group'])
transactions_campaign_id_df = pd.merge(transactions_campaign_id_df, campaigns_trimmed_df, left_on='campaign_id', right_on='campaign_id', how='left')

# Defining date range of interest for each transaction
date_start = transactions_campaign_id_df['campaign_date']
date_range = pd.Timedelta(days=10)
end_date = date_start + date_range

# Filter to transactions that occurred within 10 days of the campaign start and only to targeted moviegoers
campaign_target_total_spend = transactions_campaign_id_df.loc[(transactions_campaign_id_df['transaction_line_date'] >= date_start) & (transactions_campaign_id_df['transaction_line_date'] <= end_date)]
campaign_target_total_spend = campaign_target_total_spend.loc[campaign_target_total_spend['target_profile_id'] == 2]

# Groupby Campaign_id and sum the 'transaction_line_total' for the final campaign_target_total_spend data
campaign_target_total_spend = campaign_target_total_spend.drop(columns=['moviegoer_id', 'target_profile_id', 'transaction_line_id', 'transaction_id', 'transaction_line_date', 'transaction_line_type', 'transaction_line_is_admission', 'campaign_date'])
campaign_target_total_spend = campaign_target_total_spend.groupby(['campaign_id']).sum()
campaign_target_total_spend.rename(columns = {'transaction_line_total':'campaign_target_total_spend'}, inplace=True)

# Merge campaign_target_total_spend data with the campaigns_summary_df and fill NaNs with 0
campaigns_summary_df = pd.merge(campaigns_summary_df, campaign_target_total_spend, left_on='campaign_id', right_index=True, how='left')
campaigns_summary_df['campaign_target_total_spend'] = campaigns_summary_df['campaign_target_total_spend'].fillna(0)
campaigns_summary_df.head()

,campaign_id,campaign_description,campaign_date,campaign_target_group_count,campaign_control_group_count,campaign_has_control_group,campaign_unique_email_opens,campaign_unique_link_clicks,campaign_target_total_spend
0,22160,Campaign # 1,2019-01-01,1,0,False,0,1,0.00
1,22166,Campaign # 2,2019-01-03,1981,0,False,181,30,17551.56
2,22173,Campaign # 3,2019-01-04,2648,0,False,177,83,4304.76
3,22174,Campaign # 4,2019-01-04,3540,0,False,289,217,11006.17
4,22175,Campaign # 5,2019-01-04,11194,0,False,386,94,58950.50


In [8]:
## Creating the campaign_target_total_admission count column

# Filtering for the transactions that occurred within 10 days of the start of the campaign,
#     for only moviegoer's that were targeted, and for only cases where 'transaction_line_is_admission' is True
campaign_target_total_admission = transactions_campaign_id_df.loc[(transactions_campaign_id_df['transaction_line_date'] >= date_start) & (transactions_campaign_id_df['transaction_line_date'] <= end_date)]
campaign_target_total_admission = campaign_target_total_admission.loc[campaign_target_total_admission['target_profile_id'] == 2]
campaign_target_total_admission = campaign_target_total_admission.loc[campaign_target_total_admission['transaction_line_is_admission'] == True]

# Formatting table and dropping columns for .groupby() and .count() functions before merging with summary table.
campaign_target_total_admission = campaign_target_total_admission.drop(columns=['moviegoer_id','target_profile_id','transaction_line_id','transaction_id','transaction_line_total','transaction_line_date','transaction_line_type', 'campaign_date'])
campaign_target_total_admission = campaign_target_total_admission.groupby(['campaign_id']).count()
campaign_target_total_admission.rename(columns = {'transaction_line_is_admission':'campaign_target_total_admission'}, inplace=True)

# Merging campaign_target_total_admission final table with the campaigns_summary_df using left join.
campaigns_summary_df = pd.merge(campaigns_summary_df, campaign_target_total_admission, left_on='campaign_id', right_index=True, how='left')
campaigns_summary_df['campaign_target_total_admission'] = campaigns_summary_df['campaign_target_total_admission'].fillna(0).astype(int)
campaigns_summary_df.head()

,campaign_id,campaign_description,campaign_date,campaign_target_group_count,campaign_control_group_count,campaign_has_control_group,campaign_unique_email_opens,campaign_unique_link_clicks,campaign_target_total_spend,campaign_target_total_admission
0,22160,Campaign # 1,2019-01-01,1,0,False,0,1,0.00,0
1,22166,Campaign # 2,2019-01-03,1981,0,False,181,30,17551.56,1071
2,22173,Campaign # 3,2019-01-04,2648,0,False,177,83,4304.76,320
3,22174,Campaign # 4,2019-01-04,3540,0,False,289,217,11006.17,675
4,22175,Campaign # 5,2019-01-04,11194,0,False,386,94,58950.50,3382


In [9]:
## Creating the campaign_target_box_office_spend column

# Filtering for the transactions that occurred within 10 days of the start of the campaign,
#     for only moviegoer's that were targeted, and for only cases where transaction_line_type is box_office
campaign_target_box_office_spend = transactions_campaign_id_df.loc[(transactions_campaign_id_df['transaction_line_date'] >= date_start) & (transactions_campaign_id_df['transaction_line_date'] <= end_date)]
campaign_target_box_office_spend = campaign_target_box_office_spend.loc[campaign_target_box_office_spend['target_profile_id'] == 2]
campaign_target_box_office_spend = campaign_target_box_office_spend.loc[campaign_target_box_office_spend['transaction_line_type'] == 'box_office']

# Formatting table and dropping columns for .groupby() and .count() functions before merging with summary table.
campaign_target_box_office_spend = campaign_target_box_office_spend.drop(columns=['moviegoer_id', 'target_profile_id', 'transaction_line_id', 'transaction_id', 'transaction_line_date', 'transaction_line_type', 'transaction_line_is_admission', 'campaign_date'])
campaign_target_box_office_spend = campaign_target_box_office_spend.groupby(['campaign_id']).sum()
campaign_target_box_office_spend = campaign_target_box_office_spend.rename(columns={'transaction_line_total':'campaign_target_box_office_spend'})

# Merging campaign_target_box_office_spend final table with the campaigns_summary_df using left join and filling NaN values with 0.
campaigns_summary_df = pd.merge(campaigns_summary_df, campaign_target_box_office_spend, left_on='campaign_id', right_index=True, how='left')
campaigns_summary_df['campaign_target_box_office_spend'] = campaigns_summary_df['campaign_target_box_office_spend'].fillna(0).astype(float)
campaigns_summary_df.head()

,campaign_id,campaign_description,campaign_date,campaign_target_group_count,campaign_control_group_count,campaign_has_control_group,campaign_unique_email_opens,campaign_unique_link_clicks,campaign_target_total_spend,campaign_target_total_admission,campaign_target_box_office_spend
0,22160,Campaign # 1,2019-01-01,1,0,False,0,1,0.00,0,0.00
1,22166,Campaign # 2,2019-01-03,1981,0,False,181,30,17551.56,1071,10244.43
2,22173,Campaign # 3,2019-01-04,2648,0,False,177,83,4304.76,320,2850.40
3,22174,Campaign # 4,2019-01-04,3540,0,False,289,217,11006.17,675,7324.09
4,22175,Campaign # 5,2019-01-04,11194,0,False,386,94,58950.50,3382,33896.70


In [10]:
## Creating the campaign_target_concession_spend column

# Filtering for the transactions that occurred within 10 days of the start of the campaign,
#     for only moviegoer's that were targeted, and for only cases where transaction_line_type is 'concession'
campaign_target_concession_spend = transactions_campaign_id_df.loc[(transactions_campaign_id_df['transaction_line_date'] >= date_start) & (transactions_campaign_id_df['transaction_line_date'] <= end_date)]
campaign_target_concession_spend = campaign_target_concession_spend.loc[campaign_target_concession_spend['target_profile_id'] == 2]
campaign_target_concession_spend = campaign_target_concession_spend.loc[campaign_target_concession_spend['transaction_line_type'] == 'concession']

# Formatting table and dropping columns for .groupby() and .sum() functions,
#     to get a total sum of concession spending by targeted moviegoers.
campaign_target_concession_spend = campaign_target_concession_spend.drop(columns=['moviegoer_id', 'target_profile_id', 'transaction_line_id', 'transaction_id', 'transaction_line_date', 'transaction_line_type', 'transaction_line_is_admission', 'campaign_date'])
campaign_target_concession_spend = campaign_target_concession_spend.groupby(['campaign_id']).sum()
campaign_target_concession_spend = campaign_target_concession_spend.rename(columns={'transaction_line_total':'campaign_target_concession_spend'})


# Merging campaign_target_concession_spend final table with the campaigns_summary_df using left join,
#     and filling NaN values with 0.
campaigns_summary_df = pd.merge(campaigns_summary_df, campaign_target_concession_spend, left_on='campaign_id', right_index=True, how='left')
campaigns_summary_df['campaign_target_concession_spend'] = campaigns_summary_df['campaign_target_concession_spend'].fillna(0).astype(float)
campaigns_summary_df.head()

,campaign_id,campaign_description,campaign_date,campaign_target_group_count,campaign_control_group_count,campaign_has_control_group,campaign_unique_email_opens,campaign_unique_link_clicks,campaign_target_total_spend,campaign_target_total_admission,campaign_target_box_office_spend,campaign_target_concession_spend
0,22160,Campaign # 1,2019-01-01,1,0,False,0,1,0.00,0,0.00,0.00
1,22166,Campaign # 2,2019-01-03,1981,0,False,181,30,17551.56,1071,10244.43,7000.88
2,22173,Campaign # 3,2019-01-04,2648,0,False,177,83,4304.76,320,2850.40,1154.36
3,22174,Campaign # 4,2019-01-04,3540,0,False,289,217,11006.17,675,7324.09,2705.83
4,22175,Campaign # 5,2019-01-04,11194,0,False,386,94,58950.50,3382,33896.70,23952.55


In [11]:
## Creating the campaign_control_total_spend column

# Filter to transactions that occurred within 10 days of the campaign start and only to targeted moviegoers
campaign_control_total_spend = transactions_campaign_id_df.loc[(transactions_campaign_id_df['transaction_line_date'] >= date_start) & (transactions_campaign_id_df['transaction_line_date'] <= end_date)]
campaign_control_total_spend = campaign_control_total_spend.loc[campaign_control_total_spend['target_profile_id'] == 3]

# Groupby Campaign_id and sum the 'transaction_line_total' for the final campaign_target_total_spend data
campaign_control_total_spend = campaign_control_total_spend.drop(columns=['moviegoer_id', 'target_profile_id', 'transaction_line_id', 'transaction_id', 'transaction_line_date', 'transaction_line_type', 'transaction_line_is_admission', 'campaign_date'])
campaign_control_total_spend = campaign_control_total_spend.groupby(['campaign_id']).sum()
campaign_control_total_spend.rename(columns = {'transaction_line_total':'campaign_control_total_spend'}, inplace=True)

# Merge campaign_target_total_spend data with the campaigns_summary_df and fill NaNs with 0
campaigns_summary_df = pd.merge(campaigns_summary_df, campaign_control_total_spend, left_on='campaign_id', right_index=True, how='left')
campaigns_summary_df['campaign_control_total_spend'] = campaigns_summary_df['campaign_control_total_spend'].fillna(0).astype(float)
campaigns_summary_df.head()

,campaign_id,campaign_description,campaign_date,campaign_target_group_count,campaign_control_group_count,campaign_has_control_group,campaign_unique_email_opens,campaign_unique_link_clicks,campaign_target_total_spend,campaign_target_total_admission,campaign_target_box_office_spend,campaign_target_concession_spend,campaign_control_total_spend
0,22160,Campaign # 1,2019-01-01,1,0,False,0,1,0.00,0,0.00,0.00,0.00
1,22166,Campaign # 2,2019-01-03,1981,0,False,181,30,17551.56,1071,10244.43,7000.88,0.00
2,22173,Campaign # 3,2019-01-04,2648,0,False,177,83,4304.76,320,2850.40,1154.36,360.15
3,22174,Campaign # 4,2019-01-04,3540,0,False,289,217,11006.17,675,7324.09,2705.83,1075.51
4,22175,Campaign # 5,2019-01-04,11194,0,False,386,94,58950.50,3382,33896.70,23952.55,1972.26


In [12]:
## Creating the campaign_control_total_admission count column

# Filtering for the transactions that occurred within 10 days of the start of the campaign,
#     for only moviegoer's that were targeted, and for only cases where 'transaction_line_is_admission' is True
campaign_control_total_admission = transactions_campaign_id_df.loc[(transactions_campaign_id_df['transaction_line_date'] >= date_start) & (transactions_campaign_id_df['transaction_line_date'] <= end_date)]
campaign_control_total_admission = campaign_control_total_admission.loc[campaign_control_total_admission['target_profile_id'] == 3]
campaign_control_total_admission = campaign_control_total_admission.loc[campaign_control_total_admission['transaction_line_is_admission'] == True]

# Formatting table and dropping columns for .groupby() and .count() functions before merging with summary table.
campaign_control_total_admission = campaign_control_total_admission.drop(columns=['moviegoer_id','target_profile_id','transaction_line_id','transaction_id','transaction_line_total','transaction_line_date','transaction_line_type', 'campaign_date'])
campaign_control_total_admission = campaign_control_total_admission.groupby(['campaign_id']).count()
campaign_control_total_admission = campaign_control_total_admission.rename(columns = {'transaction_line_is_admission':'campaign_control_total_admission'})

# Merging campaign_target_total_admission final table with the campaigns_summary_df using left join.
campaigns_summary_df = pd.merge(campaigns_summary_df, campaign_control_total_admission, left_on='campaign_id', right_index=True, how='left')
campaigns_summary_df['campaign_control_total_admission'] = campaigns_summary_df['campaign_control_total_admission'].fillna(0).astype(int)
campaigns_summary_df.head()

,campaign_id,campaign_description,campaign_date,campaign_target_group_count,campaign_control_group_count,campaign_has_control_group,campaign_unique_email_opens,campaign_unique_link_clicks,campaign_target_total_spend,campaign_target_total_admission,campaign_target_box_office_spend,campaign_target_concession_spend,campaign_control_total_spend,campaign_control_total_admission
0,22160,Campaign # 1,2019-01-01,1,0,False,0,1,0.00,0,0.00,0.00,0.00,0
1,22166,Campaign # 2,2019-01-03,1981,0,False,181,30,17551.56,1071,10244.43,7000.88,0.00,0
2,22173,Campaign # 3,2019-01-04,2648,0,False,177,83,4304.76,320,2850.40,1154.36,360.15,28
3,22174,Campaign # 4,2019-01-04,3540,0,False,289,217,11006.17,675,7324.09,2705.83,1075.51,63
4,22175,Campaign # 5,2019-01-04,11194,0,False,386,94,58950.50,3382,33896.70,23952.55,1972.26,121


In [13]:
## Creating the campaign_target_spend_per_moviegoer column

# Filtering for the transactions that occurred within 10 days of the start of the campaign,
#     for only moviegoer's that were targeted, and for only cases where 'transaction_line_is_admission' is True
campaign_target_spend = transactions_campaign_id_df.loc[(transactions_campaign_id_df['transaction_line_date'] >= date_start) & (transactions_campaign_id_df['transaction_line_date'] <= end_date)]
campaign_target_spend = campaign_target_spend.loc[campaign_target_spend['target_profile_id'] == 2]

# Determining the number of targeted moviegoers per campaign
targeted_moviegoers_per_campaign = targeted_moviegoers_df.groupby(['campaign_id']).nunique()['moviegoer_id']
targeted_moviegoers_per_campaign = pd.DataFrame(targeted_moviegoers_per_campaign)
targeted_moviegoers_per_campaign = targeted_moviegoers_per_campaign.rename(columns={'moviegoer_id':'unique_targeted_moviegoers'})


# Groupby campaign_id and sum the 'transaction_line_total' for the final campaign_target_total_spend data
campaign_target_spend = campaign_target_spend.drop(columns=['moviegoer_id', 'target_profile_id', 'transaction_line_id', 'transaction_id', 'transaction_line_date', 'transaction_line_type', 'transaction_line_is_admission', 'campaign_date'])
campaign_target_spend = campaign_target_spend.groupby(['campaign_id']).sum()
campaign_target_spend = campaign_target_spend.rename(columns = {'transaction_line_total':'campaign_target_spend'})

# Merging the targeted_moviegoers_per_campaign with the campaign_target_spend totals to then divide 
#     for target_spend_per_moviegoer using 'campaign_target_spend' column / 'unique_targeted_moviegoers' column
  
campaign_target_spend_per_moviegoer = pd.merge(campaign_target_spend, targeted_moviegoers_per_campaign, left_on='campaign_id', right_index=True, how='left')
campaign_target_spend_per_moviegoer['campaign_target_spend_per_moviegoer'] = campaign_target_spend_per_moviegoer['campaign_target_spend'] / campaign_target_spend_per_moviegoer['unique_targeted_moviegoers']
campaign_target_spend_per_moviegoer['campaign_target_spend_per_moviegoer'] = campaign_target_spend_per_moviegoer['campaign_target_spend_per_moviegoer'].round(decimals=2)

# Dropping excess columns used to determine the campaign_target_spend_per_moviegoer before merging with main summary df.
campaign_target_spend_per_moviegoer = campaign_target_spend_per_moviegoer.drop(columns=['campaign_target_spend','unique_targeted_moviegoers'])

# Merge the 'campaign_target_spend_per_moviegoer' column into the main 'campaigns_summary_df' using LEFT JOIN.
campaigns_summary_df = pd.merge(campaigns_summary_df, campaign_target_spend_per_moviegoer, left_on='campaign_id', right_index=True, how='left')
campaigns_summary_df['campaign_target_spend_per_moviegoer'] = campaigns_summary_df['campaign_target_spend_per_moviegoer'].fillna(0).astype(float)
campaigns_summary_df.head()

,campaign_id,campaign_description,campaign_date,campaign_target_group_count,campaign_control_group_count,campaign_has_control_group,campaign_unique_email_opens,campaign_unique_link_clicks,campaign_target_total_spend,campaign_target_total_admission,campaign_target_box_office_spend,campaign_target_concession_spend,campaign_control_total_spend,campaign_control_total_admission,campaign_target_spend_per_moviegoer
0,22160,Campaign # 1,2019-01-01,1,0,False,0,1,0.00,0,0.00,0.00,0.00,0,0.00
1,22166,Campaign # 2,2019-01-03,1981,0,False,181,30,17551.56,1071,10244.43,7000.88,0.00,0,8.86
2,22173,Campaign # 3,2019-01-04,2648,0,False,177,83,4304.76,320,2850.40,1154.36,360.15,28,1.44
3,22174,Campaign # 4,2019-01-04,3540,0,False,289,217,11006.17,675,7324.09,2705.83,1075.51,63,2.83
4,22175,Campaign # 5,2019-01-04,11194,0,False,386,94,58950.50,3382,33896.70,23952.55,1972.26,121,5.10


In [14]:
## Creating the campaign_control_spend_per_moviegoer column

# Filtering for the transactions that occurred within 10 days of the start of the campaign,
#     for only moviegoer's that were targeted, and for only cases where 'transaction_line_is_admission' is True
campaign_control_spend = transactions_campaign_id_df.loc[(transactions_campaign_id_df['transaction_line_date'] >= date_start) & (transactions_campaign_id_df['transaction_line_date'] <= end_date)]
campaign_control_spend = campaign_control_spend.loc[campaign_control_spend['target_profile_id'] == 3]

# Determining the number of targeted moviegoers per campaign
control_moviegoers_per_campaign = targeted_moviegoers_df.groupby(['campaign_id']).nunique()['moviegoer_id']
control_moviegoers_per_campaign = pd.DataFrame(control_moviegoers_per_campaign)
control_moviegoers_per_campaign = control_moviegoers_per_campaign.rename(columns={'moviegoer_id':'unique_control_moviegoers'})


# Groupby campaign_id and sum the 'transaction_line_total' for the final campaign_control_total_spend data
campaign_control_spend = campaign_control_spend.drop(columns=['moviegoer_id', 'target_profile_id', 'transaction_line_id', 'transaction_id', 'transaction_line_date', 'transaction_line_type', 'transaction_line_is_admission', 'campaign_date'])
campaign_control_spend = campaign_control_spend.groupby(['campaign_id']).sum()
campaign_control_spend = campaign_control_spend.rename(columns = {'transaction_line_total':'campaign_control_spend'})

# Merging the control_moviegoers_per_campaign with the campaign_control_spend totals to then divide 
#     for control_spend_per_moviegoer using 'campaign_control_spend' column / 'unique_control_moviegoers' column
  
campaign_control_spend_per_moviegoer = pd.merge(campaign_control_spend, control_moviegoers_per_campaign, left_on='campaign_id', right_index=True, how='left')
campaign_control_spend_per_moviegoer['campaign_control_spend_per_moviegoer'] = campaign_control_spend_per_moviegoer['campaign_control_spend'] / campaign_control_spend_per_moviegoer['unique_control_moviegoers']
campaign_control_spend_per_moviegoer['campaign_control_spend_per_moviegoer'] = campaign_control_spend_per_moviegoer['campaign_control_spend_per_moviegoer'].round(decimals=2)


# # Dropping excess columns used to determine the campaign_target_spend_per_moviegoer before merging with main summary df.
campaign_control_spend_per_moviegoer = campaign_control_spend_per_moviegoer.drop(columns=['campaign_control_spend','unique_control_moviegoers'])

# # Merge the 'campaign_target_spend_per_moviegoer' column into the main 'campaigns_summary_df' using LEFT JOIN.
campaigns_summary_df = pd.merge(campaigns_summary_df, campaign_control_spend_per_moviegoer, left_on='campaign_id', right_index=True, how='left')
campaigns_summary_df['campaign_control_spend_per_moviegoer'] = campaigns_summary_df['campaign_control_spend_per_moviegoer'].fillna(0).astype(float)
campaigns_summary_df.head()

,campaign_id,campaign_description,campaign_date,campaign_target_group_count,campaign_control_group_count,campaign_has_control_group,campaign_unique_email_opens,campaign_unique_link_clicks,campaign_target_total_spend,campaign_target_total_admission,campaign_target_box_office_spend,campaign_target_concession_spend,campaign_control_total_spend,campaign_control_total_admission,campaign_target_spend_per_moviegoer,campaign_control_spend_per_moviegoer
0,22160,Campaign # 1,2019-01-01,1,0,False,0,1,0.00,0,0.00,0.00,0.00,0,0.00,0.00
1,22166,Campaign # 2,2019-01-03,1981,0,False,181,30,17551.56,1071,10244.43,7000.88,0.00,0,8.86,0.00
2,22173,Campaign # 3,2019-01-04,2648,0,False,177,83,4304.76,320,2850.40,1154.36,360.15,28,1.44,0.12
3,22174,Campaign # 4,2019-01-04,3540,0,False,289,217,11006.17,675,7324.09,2705.83,1075.51,63,2.83,0.28
4,22175,Campaign # 5,2019-01-04,11194,0,False,386,94,58950.50,3382,33896.70,23952.55,1972.26,121,5.10,0.17


In [15]:
## Creating the campaign_spend_uplift_per_moviegoer and campaign_spend_uplift_per_moviegoer_% columns

# Merging the campaign_target_spend_per_moviegoer with the campaign_control_spend_per_moviegoer for calculations
campaign_spend_uplift_per_moviegoer = pd.merge(campaign_target_spend_per_moviegoer, campaign_control_spend_per_moviegoer, left_on='campaign_id', right_index=True, how='left')
campaign_spend_uplift_per_moviegoer['campaign_control_spend_per_moviegoer'] = campaign_spend_uplift_per_moviegoer['campaign_control_spend_per_moviegoer'].fillna(0).astype(float)

# Calculating uplift per moviegoer for each campaign by subtracting
#     the 'campaign_target_total_spend_per_moviegoer' - 'campaign_control_total_spend_per_moviegoer'.
#     Calulated the % of uplift per moviegoer by using the 'campaign_spend_uplift_per_moviegoer' dollar increase
#     and dividing by the control group spending per moviegoer, then multiplying by 100 to get a percentage.
#     Also, formatted inf values into 0.0s. 
campaign_spend_uplift_per_moviegoer['campaign_spend_uplift_per_moviegoer'] = campaign_spend_uplift_per_moviegoer['campaign_target_spend_per_moviegoer'] - campaign_spend_uplift_per_moviegoer['campaign_control_spend_per_moviegoer']
campaign_spend_uplift_per_moviegoer['campaign_spend_uplift_per_moviegoer_%'] = (campaign_spend_uplift_per_moviegoer['campaign_spend_uplift_per_moviegoer'] / campaign_spend_uplift_per_moviegoer['campaign_control_spend_per_moviegoer'] * 100)
campaign_spend_uplift_per_moviegoer['campaign_spend_uplift_per_moviegoer_%'] = campaign_spend_uplift_per_moviegoer['campaign_spend_uplift_per_moviegoer_%'].replace([np.inf, -np.inf], 0.0)
campaign_spend_uplift_per_moviegoer['campaign_spend_uplift_per_moviegoer_%'] = campaign_spend_uplift_per_moviegoer['campaign_spend_uplift_per_moviegoer_%'].round(decimals=2)

# Dropped columns used for calculations before merging with campaigns_summary_df table
campaign_spend_uplift_per_moviegoer = campaign_spend_uplift_per_moviegoer.drop(columns=['campaign_target_spend_per_moviegoer', 'campaign_control_spend_per_moviegoer'])

# Merged the calculated uplift columns into 'campaigns_summary_df' and formatted NaN values to 0.00
campaigns_summary_df = pd.merge(campaigns_summary_df, campaign_spend_uplift_per_moviegoer, left_on='campaign_id', right_index=True, how='left')
campaigns_summary_df['campaign_spend_uplift_per_moviegoer'] = campaigns_summary_df['campaign_spend_uplift_per_moviegoer'].fillna(0).astype(float)
campaigns_summary_df['campaign_spend_uplift_per_moviegoer_%'] = campaigns_summary_df['campaign_spend_uplift_per_moviegoer_%'].fillna(0).astype(float)
campaigns_summary_df.head()

,campaign_id,campaign_description,campaign_date,campaign_target_group_count,campaign_control_group_count,campaign_has_control_group,campaign_unique_email_opens,campaign_unique_link_clicks,campaign_target_total_spend,campaign_target_total_admission,campaign_target_box_office_spend,campaign_target_concession_spend,campaign_control_total_spend,campaign_control_total_admission,campaign_target_spend_per_moviegoer,campaign_control_spend_per_moviegoer,campaign_spend_uplift_per_moviegoer,campaign_spend_uplift_per_moviegoer_%
0,22160,Campaign # 1,2019-01-01,1,0,False,0,1,0.00,0,0.00,0.00,0.00,0,0.00,0.00,0.00,0.00
1,22166,Campaign # 2,2019-01-03,1981,0,False,181,30,17551.56,1071,10244.43,7000.88,0.00,0,8.86,0.00,8.86,0.00
2,22173,Campaign # 3,2019-01-04,2648,0,False,177,83,4304.76,320,2850.40,1154.36,360.15,28,1.44,0.12,1.32,1100.00
3,22174,Campaign # 4,2019-01-04,3540,0,False,289,217,11006.17,675,7324.09,2705.83,1075.51,63,2.83,0.28,2.55,910.71
4,22175,Campaign # 5,2019-01-04,11194,0,False,386,94,58950.50,3382,33896.70,23952.55,1972.26,121,5.10,0.17,4.93,2900.00


In [16]:
## Saving 'campaigns_summary_df' as a CSV into same folder as data.
campaigns_summary_df.to_csv('monthly_campaigns_summary.csv')